In [1]:
!pip install requests bs4 graphviz pymorphy3 googletrans

In [2]:
import requests
from bs4 import BeautifulSoup
import json
from graphviz import Digraph
import pymorphy3
from googletrans import Translator

In [3]:
# Инициализация морфологического анализатора и переводчика.
morph = pymorphy3.MorphAnalyzer()
translator = Translator()

# MediaWiki API URL.
WIKI_API_URL = "https://ru.wikipedia.org/w/api.php"

In [4]:
# Функция для поиска статьи и получения её контента.
def get_wiki_page_content(title):
    params = {
        "action": "query",
        "prop": "extracts|links|images",
        "titles": title,
        "format": "json",
        "explaintext": 1,
        "pllimit": "max",
        "imlimit": "max"
    }
    response = requests.get(WIKI_API_URL, params=params)
    data = response.json()
    pages = data['query']['pages']

    # Извлекаем первую страницу (она единственная в ответе).
    page = next(iter(pages.values()))

    # Получаем основной текст и связанные узлы (ссылки).
    content = {
        "title": page.get("title"),
        "text": page.get("extract", ""),
        "links": [link['title'] for link in page.get("links", [])],
        "images": [image['title'] for image in page.get("images", [])]
    }

    return content

In [5]:
# Функция для построения графа в формате DOT.
def build_dot_graph(data):
    dot = Digraph(comment='Wikimedia Articles Graph')

    # Добавляем узлы и связи.
    for node, content in data.items():
        dot.node(node, node)
        for link in content['links']:
            if link in data:  # Добавляем связь только с известными узлами.
                dot.edge(node, link)

    return dot

In [6]:
# Функция для сбора данных и создания графа.
def collect_wiki_data(subject, limit=30):
    visited = set()
    to_visit = [subject]
    articles_data = {}

    while to_visit and len(articles_data) < limit:
        current_subject = to_visit.pop(0)
        if current_subject in visited:
            continue

        print(f"Fetching article: {current_subject}")
        content = get_wiki_page_content(current_subject)
        articles_data[current_subject] = content
        visited.add(current_subject)

        # Добавляем ссылки на другие статьи в очередь для посещения.
        to_visit.extend([link for link in content['links'] if link not in visited and link not in to_visit])

    return articles_data

In [7]:
# Функция для автоматического перевода текста на русский (если статья на другом языке).
def translate_text(text, dest='ru'):
    translated = translator.translate(text, dest=dest)
    return translated.text

In [10]:
subject = "Чёрная металлургия"  # Начальная статья.
articles_data = collect_wiki_data(subject)

# Сохраняем данные в формате JSON.
with open('wiki_data.json', 'w', encoding='utf-8') as f:
    json.dump(articles_data, f, ensure_ascii=False, indent=4)

# Создаем граф в формате DOT.
dot_graph = build_dot_graph(articles_data)

# Сохраняем граф.
dot_graph.render('wiki_graph.dot', view=True)

Fetching article: Чёрная металлургия
Fetching article: Arcelor Mittal
Fetching article: Gerdau
Fetching article: POSCO
Fetching article: Tata Steel
Fetching article: Techint
Fetching article: U.S. Steel
Fetching article: Wayback Machine
Fetching article: Австралия
Fetching article: Агломерат (металлургия)
Fetching article: Агломерационная машина
Fetching article: Агломерационная фабрика
Fetching article: Агломерация (металлургия)
Fetching article: Азовсталь
Fetching article: Алюминиевая промышленность
Fetching article: Алюминотермия
Fetching article: Аносов, Павел Петрович
Fetching article: Антрацит
Fetching article: Аргентина
Fetching article: Аффинаж
Fetching article: Байков, Александр Александрович
Fetching article: Бардин, Иван Павлович
Fetching article: Бессемер, Генри
Fetching article: Бессемеровский процесс
Fetching article: Блюминг (стан)
Fetching article: Брикет (металлургия)
Fetching article: Брикетный пресс
Fetching article: Вакууматор
Fetching article: Вакуумная металлургия

'wiki_graph.dot.pdf'